<a href="https://colab.research.google.com/github/Black-fox17/DSA/blob/main/T_of_Ray_Colab_ocr_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Program

In [ ]:
import ipywidgets as widgets
from IPython.display import display


In [ ]:
!python -m pip install paddlepaddle-gpu # -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install "paddleocr>=2.0.1" # Recommend to use version 2.0.1+
!pip install easyocr


  Using cached easyocr-1.7.1-py3-none-any.whl (2.9 MB)
  Using cached python_bidi-0.4.2-py2.py3-none-any.whl (30 kB)
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Usi

In [ ]:
import easyocr


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## video output

##helper


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Define function to check if text color is green
def is_green_text(roi):
    # Convert the region to HSV color space
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define range for green color in HSV
    lower_green = np.array([50, 100, 100])  # Lower bound for green color
    upper_green = np.array([80, 255, 255])  # Upper bound for green color

    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv_roi, lower_green, upper_green)

    # Check if any green pixels exist within the mask
    return cv2.countNonZero(mask) > 0

In [ ]:
from PIL import Image

def enlarge_region_of_interest(region_of_interest, scale_factor):
    """
    Resize a specific region of interest (ROI) within an image by a given scale factor.

    Args:
        region_of_interest (PIL.Image.Image): Region of interest (ROI) as a PIL Image.
        scale_factor (float): Factor by which to resize the ROI.

    Returns:
        PIL.Image.Image: Resized image of the ROI.
    """
    # Get the original dimensions of the ROI
    width, height = region_of_interest.size

    # Calculate the new dimensions after resizing
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)

    # Resize the ROI
    resized_roi = region_of_interest.resize((new_width, new_height), Image.ANTIALIAS)

    return resized_roi

# Now you can use the resized_roi for further processing or visualization


In [ ]:
def process_text_and_roi(ch_text, image):
    """
    Process text elements and regions of interest (ROIs) within an image.

    Args:
        ch_text (list): List of text elements, each containing (box, text, confidence).
        image (numpy.ndarray): Image array (e.g., from video frame) containing regions of interest.

    Returns:
        list: Updated list of text elements after processing.
    """
    modified_ch_text = []

    for i in range(len(ch_text)):
        # Extract the current text element
        box, text, _ = ch_text[i]

        # Check if the current text contains "竞份涨幅"
        if "竞价涨幅" in text:
            # Look for the next text element
            if i + 1 < len(ch_text):
                _, next_text, _ = ch_text[i + 1]

                # Find the position of the last digit in the next text
                last_digit_index = len(next_text) - 1
                while last_digit_index >= 0:
                    if next_text[last_digit_index].isdigit():
                        break
                    last_digit_index -= 1

                # Replace the last digit with "%" symbol
                if last_digit_index >= 0:
                    modified_number = next_text[:last_digit_index] + "%"  # Replace the last digit with "%"
                    next_text = modified_number

                # Update the next text in the ch_text list
                ch_text[i + 1] = (ch_text[i + 1][0], next_text, ch_text[i + 1][2])

        # Extract the bounding box coordinates
        x_min, y_min = box[0]
        x_max, y_max = box[2]
        # Extract the region within the bounding box from the image
        roi = image[int(y_min):int(y_max), int(x_min):int(x_max)]

        # Check if the text color is green (example function placeholder)
        is_green = is_green_text(roi)

        # If the text color is green and doesn't already contain a hyphen, add it
        if is_green and "-" not in text:
            text = "-" + text

        # Append the modified text element with updated bounding box to the list
        modified_ch_text.append((box, text, _))

    return modified_ch_text

# Helper function to determine if a line contains only Chinese characters
def is_chinese(text):
    return not bool(re.search(r'\d', text))


In [ ]:
def preprocess_for_ocr(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Enhance image contrast using histogram equalization

    # Apply adaptive thresholding to binarize the image

    return gray

In [ ]:
import cv2
import easyocr
from paddleocr import PaddleOCR

def detect_digit_from_ocr(roi):
    # Extract the bottom-right corner of the ROI (100x100 pixels)
    corner_size = 100
    bottom_right_corner = roi[-corner_size:, -corner_size:]
    cropped_corner = bottom_right_corner[:, -80:]

    # Initialize OCR readers
    name_reader = easyocr.Reader(['en'])
    reader_en_ch = PaddleOCR(use_angle_cls=True, lang='en')

    # Perform OCR on the bottom-right corner using EasyOCR
    x = name_reader.readtext(cropped_corner)
    # Perform OCR on the bottom-right corner using PaddleOCR
    y = reader_en_ch.ocr(cropped_corner)

    # Extract text results from OCR outputs
    x_text = x[0][1] if x else ''

    # Handle PaddleOCR result
    if y and isinstance(y, list) and len(y) > 0 and isinstance(y[0], list) and len(y[0]) > 0 and isinstance(y[0][0], list) and len(y[0][0]) > 1:
        y_text = y[0][0][1][0]
    else:
        y_text = ''

    # Check for specific characters in OCR results
    if '7' in x_text or '/' in x_text or '7' in y_text or '/' in y_text:
      if is_green_text(cropped_corner):
        return '-7'
      else:
        return '7'
    elif '4' in x_text or '4' in y_text:
      if is_green_text(cropped_corner):
        return '-4'
      else:
        return '4'
    else:
      if is_green_text(cropped_corner):
        return '-1'
      else:
        return '1'


In [ ]:
def clean_and_convert_to_float(s):
    # Check if the input is a string or convertible to a string
    if not isinstance(s, (str, int, float)):
        return None  # Return None if the input is not a string, integer, or float

    # Convert input to string if it's not already a string
    if not isinstance(s, str):
        s = str(s)

    # Check if the input string is empty or does not contain valid numeric characters
    if s.strip() == '' or not any(char.isdigit() or char == '.' for char in s):
        return None  # Return None if the input is empty or not a valid number

    # Remove non-numeric characters from the string
    cleaned_string = ''.join(char for char in s if char.isdigit() or char == '.' or char =='-')

    # Convert the cleaned string to a float
    try:
        result = float(cleaned_string)
        return result
    except ValueError:
        # Handle the case where conversion to float fails
        return None  # Return None if conversion fails


In [ ]:
def split_video_into_chunks(video_path, num_chunks=20):
    capture = cv2.VideoCapture(video_path)
    total_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the number of frames per chunk
    frames_per_chunk = total_frames // num_chunks

    chunk_frames_list = []

    for i in range(num_chunks):
        # Calculate start frame and end frame for the current chunk
        start_frame = i * frames_per_chunk
        end_frame = (i + 1) * frames_per_chunk - 1

        # For the last chunk, adjust the end frame to ensure it covers all remaining frames
        if i == num_chunks - 1:
            end_frame = total_frames - 1

        # Append (start_frame, end_frame) tuple to the list
        chunk_frames_list.append((start_frame, end_frame))

    # Release the initial VideoCapture object used for splitting
    capture.release()

    return chunk_frames_list

In [ ]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
from google.colab import files
def process_column(value):
    # Check if the string ends with '.' or ','
    try:
      if value.endswith('.') or value.endswith(','):
          # Replace '.' or ',' at the end with an empty string
          return value[:-1]  # Slice to remove the last character
      else:
          return value
    except:
      pass
def process_data(df):
    for column in df.columns:
        if df[column].dtype == 'O':  # Check if the column contains object (string) dtype
            df[column] = df[column].str.replace('叫:', '09:')
            df[column] = df[column].str.replace('叫', '.01')
            df[column] = df[column].str.replace('万', '')
    for column in df.columns:
        if df[column].dtype == 'O':  # Check if the column contains object (string) dtype
            # Replace '--' with '-'
            df[column] = df[column].str.replace('--', '-')

            # Replace last '9' or '96' in '竞价涨幅' column with '%'
            if '竞价涨幅' in column:
                df[column] = df[column].str.replace('9$', '%', regex=True)
                df[column] = df[column].str.replace('96$', '%', regex=True)
            elif'匹配价' in column:
              df[column] = df[column].str.replace('-', '', regex=True)
            elif '万'  in column:
              df[column] = df[column].apply(process_column)
    categories = [
    'Location',
    '匹配价',
    '竞价涨幅',
    '竞价量',
    '竞价金额(万)',
    '未匹配量',
    '未匹配金额',
    '价格',
    '均价',
    '涨跌',
    '涨幅',
    '成交量',
    '成交金额'
    ]
    all_columns = df.columns.tolist()

    # Sort columns based on categories and number suffixes
    sorted_columns = ['stock code']
    for number in range(len(all_columns)):  # Assuming columns go from 1 to 16
        for category in categories:
            column_name = f'{category}{number}'
            if column_name in all_columns:
                sorted_columns.append(column_name)

    # Reorder DataFrame columns based on sorted_columns

    df = df[sorted_columns]
    return df



def extract_ordered_values(row):
    time_sequence = ["09:15:00",'09:15:01','09:15:02','09:15:03','09:15:04','09:15:05','09:15:06','09:15:07','09:15:08','09:15:09','09:15:10','09:15:11','09:15:12','09:15:13','09:15:14','09:15:15','09:15:16','09:15:17','09:15:18','09:15:19','09:15:20','09:15:21','09:15:22','09:15:23','09:15:24','09:15:25','09:15:26','09:15:27','09:15:28','09:15:29','09:15:30','09:16:39','09:16:55','09:17:33',
                    "09:24:39",'09:24:40','09:24:41','09:24:42','09:24:43','09:24:44','09:24:45','09:24:46','09:24:47','09:24:48','09:24:49','09:24:50',
                    '09:24:51','09:24:52','09:24:53','09:24:54','09:24:55','09:24:56','09:24:57','09:24:58','09:24:59','09:25:00','09:25:01','09:25:02',
                    "14:59:33","14:59:40","14:59:41","14:59:42","14:59:43","14:59:44","14:59:45","14:59:46","14:59:47","14:59:48","14:59:49","14:59:50","14:59:51",'14:59:52','14:59:53','14:59:54','14:59:55','14:59:56','14:59:57','14:59:58','14:59:59','15:00:00','15:00:01','15:00:02','15:00:03','15:00:04','15:00:05']

    location_1 = ["09:15:00",'09:15:01','09:15:02','09:15:03','09:15:04','09:15:05','09:15:06','09:15:07','09:15:08','09:15:09','09:15:10','09:15:11','09:15:12','09:15:13','09:15:14','09:15:15','09:15:16','09:15:17','09:15:18','09:15:19','09:15:20','09:15:21','09:15:22','09:15:23','09:15:24','09:15:25','09:15:26','09:15:27','09:15:28','09:15:29','09:15:30','09:16:39','09:16:55','09:17:33']

    location_2 = ["09:24:39",'09:24:40','09:24:41','09:24:42','09:24:43','09:24:44','09:24:45','09:24:46','09:24:47','09:24:48','09:24:49','09:24:50']

    location_3 = ['09:24:51','09:24:52','09:24:53','09:24:54','09:24:55','09:24:56','09:24:57','09:24:58','09:24:59','09:25:00','09:25:01','09:25:02']

    location_4 = ["14:59:33","14:59:40","14:59:41","14:59:42","14:59:43","14:59:44","14:59:45","14:59:46","14:59:47","14:59:48","14:59:49","14:59:50","14:59:51",'14:59:52','14:59:53','14:59:54','14:59:55','14:59:56','14:59:57','14:59:58','14:59:59','15:00:00','15:00:01','15:00:02','15:00:03','15:00:04','15:00:05']


    location_lists = [location_1, location_2, location_3, location_4]
    ordered_values = []
    stock_code_raw = row['stock code']
    # Convert stock code to a numeric type (integer)
    cleaned_stock_code = ''.join(filter(str.isdigit, stock_code_raw))

    try:
        # Convert the cleaned stock code to an integer
        stock_code_int = int(cleaned_stock_code)

        # Format the integer with leading zeros (at least 6 characters wide)
        stock_code = f"{stock_code_int:06d}"
    except ValueError:
        print("Error: The cleaned stock code does not represent a valid integer.")

    # Format stock code as a six-digit zero-padded string

    for time in location_lists:
        # Find the corresponding Location column for the current time
        location_col = [col for col in row.index if 'Location' in col and row[col] in time]
        if location_col:
            location_col = location_col[0]# Get the first matching Location column

            # Extract the values from the row for the current Location column
            if len(location_col) < 10 and location_col[-1] != '0':
              values = [
                  row[location_col],  # Location Time
                  row[f'匹配价{location_col[-1]}'],  # 匹配价
                  row[f'竞价涨幅{location_col[-1]}'],  # 竞价涨幅
                  row[f'竞价量{location_col[-1]}'],  # 竞价量
                  row[f'竞价金额(万){location_col[-1]}'],  # 竞价金额(万)
                  row[f'未匹配量{location_col[-1]}'],  # 未匹配量
                  row[f'未匹配金额{location_col[-1]}']  # 未匹配金额
              ]
              ordered_values.extend(values)
            else:
              ordered_values.extend([None] * 7)  # 7 values for each time sequence
        else:
            # Append None values if the location for the current time is not found
            ordered_values.extend([None] * 7)  # 7 values for each time sequence

    # Ensure ordered_values has the correct length (28 elements for 4 times * 7 columns)
    assert len(ordered_values) == len(location_lists) * 7, "The length of ordered_values must match the expected number of elements"
    # Create a Series with the ordered values and index
    index_names = [f'Location{i}_{suffix}' for i in range(1, len(location_lists)+1)
                   for suffix in ['Time', '匹配价', '竞价涨幅', '竞价量', '竞价金额(万)', '未匹配量', '未匹配金额']]

    # Include the stock code as the first index name
    index_names.insert(0, 'stock code')

    # Combine stock code with ordered values into a Series
    ordered_values.insert(0, stock_code)
    return pd.Series(ordered_values, index=index_names)
def sorted_columns(df):
    # List of substrings to search for in column names
    search_terms = ['未匹配量', '竞价金额(万)', '竞价量', '匹配价']

    # Iterate over columns in 'ordered_df' and apply clean_and_convert_to_float to selected columns
    for col_name in df.columns:
        if any(term in col_name for term in search_terms):
            # Apply clean_and_convert_to_float to every value in the selected column
            df[col_name] = df[col_name].apply(clean_and_convert_to_float)
    # Print the updated DataFrame to verify the changes
    return df

def highlight_invalid_values(value):
    # Convert the value to string for processing
    str_value = str(value)

    # Check if the value is missing a decimal point
    if '.' not in str_value:
        return True  # Return True to indicate invalid value
    else:
        return False  # Return False for valid value

def export_to_excel(df, excel_file):
    # Export 'df' to Excel with the specified file path
    with pd.ExcelWriter(excel_file, engine='openpyxl') as excel_writer:
        df.to_excel(excel_writer, index=False, sheet_name='Sheet1')

    # Load the Excel workbook from the saved file
    wb = load_workbook(excel_file)
    ws = wb['Sheet1']

    # Define a mapping of keywords to colors for column header styling
    color_map = ['FFC7CE', 'C6EFCE', 'C9DAF8', 'FFF2CC']
    x = -1

    # Iterate over DataFrame columns to apply conditional formatting
    for col in ws.iter_cols():
        col_name = col[0].value  # Get the column header name
        if col_name and '竞价涨幅' in str(col_name):  # Check if column header contains '竞价涨幅'
            # Iterate over cells in the column (excluding header)
            for cell in col[1:]:
                if highlight_invalid_values(cell.value):
                    # Highlight cell with a specific fill color if value is invalid
                    cell.fill = PatternFill(start_color='8B0000', end_color='FFC7CE', fill_type="solid")

        if col_name and 'location' in str(col_name).lower():
            x += 1
            fill_color = color_map[x % len(color_map)]
            col_idx = df.columns.get_loc(col_name)
            col_letter = get_column_letter(col_idx + 1)
            header_cell = ws[col_letter + '1']
            header_cell.fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type="solid")
            ws.row_dimensions[header_cell.row].height = 30  # Adjust as needed for desired height

            # Color the entire column cells based on conditions
            for cell in ws[col_letter][1:]:  # Exclude header row
                if '未匹配量' in str(col_name) or '未匹配金额' in str(col_name):
                    if '-' in str(cell.value) or pd.isna(cell.value):  # Check if cell value contains '-' or is empty
                        cell.fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type="solid")
                    else:
                        cell.fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type="solid")

    # Save the updated workbook back to the Excel file
    wb.save(excel_file)

    # Download the Excel file using the files.download() function
    files.download(excel_file)

# Usage:
# df = pd.read_excel('your_file.xlsx')
# processed_df = process_data(df)
# export_to_excel(processed_df, 'processed_data.xlsx')


## code

In [ ]:
import cv2
import os
import numpy as np
import easyocr
from paddleocr import PaddleOCR,draw_ocr
import pandas as pd
import re
def get_blue_text(image):
  # Convert image to HSV color space
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  # Define the blue color range in HSV
  lower_blue = np.array([100, 150, 0])
  upper_blue = np.array([140, 255, 255])

  # Create a mask that captures areas of the image within the blue color range
  mask = cv2.inRange(hsv, lower_blue, upper_blue)

  # Find contours in the mask
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Assuming you want to crop the largest blue area
  if contours:
      # Get the largest contour
      largest_contour = max(contours, key=cv2.contourArea)

      # Get bounding box coordinates for the largest contour
      x, y, w, h = cv2.boundingRect(largest_contour)

      # Crop the image
      cropped_image = image[y:y+h, x:x+w]

  texts = reader.readtext(cropped_image, detail = 0)
  return texts[2]


def process_frames_with_ocr(video_path,start_frame, end_frame, group_name_bbox, group_elements_bbox,bounding_boxes,df=pd.DataFrame(columns=['stock code']),locations={}):
    # Define the column name for the DataFrame
    target_times = ["09:15:00",'09:15:01','09:15:02','09:15:03','09:15:04','09:15:05','09:15:06','09:15:07','09:15:08','09:15:09','09:15:10','09:15:11','09:15:12','09:15:13','09:15:14','09:15:15','09:15:16','09:15:17','09:15:18','09:15:19','09:15:20','09:15:21','09:15:22','09:15:23','09:15:24','09:15:25','09:15:26','09:15:27','09:15:28','09:15:29','09:15:30','09:16:39','09:16:55','09:17:33',
                    "09:24:39",'09:24:40','09:24:41','09:24:42','09:24:43','09:24:44','09:24:45','09:24:46','09:24:47','09:24:48','09:24:49','09:24:50',
                    '09:24:51','09:24:52','09:24:53','09:24:54','09:24:55','09:24:56','09:24:57','09:24:58','09:24:59','09:25:00','09:25:01','09:25:02',
                    "14:59:33","14:59:40","14:59:41","14:59:42","14:59:43","14:59:44","14:59:45","14:59:46","14:59:47","14:59:48","14:59:49","14:59:50","14:59:51",'14:59:52','14:59:53','14:59:54','14:59:55','14:59:56','14:59:57','14:59:58','14:59:59','15:00:00','15:00:01','15:00:02','15:00:03','15:00:04','15:00:05']

    columns = ['stock code']
    results = []
    list1 = ['Location','匹配价',"竞价涨幅","竞价量","竞价金额(万)","未匹配量","未匹配金额"]
    list2=['Location',"价格","均价","涨跌","涨幅","成交量","成交金额"]
    chinese_reader = easyocr.Reader(['ch_sim'])
    # Create an empty DataFrame with the specified column
    errors = []
    df = df
    location = locations

    reader_en_ch = PaddleOCR(use_angle_cls=True, lang='en')
    name_reader = easyocr.Reader(['en'])
    # English OCR reader
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    # Check if the video file opened successfully
    if not video_capture.isOpened():
        print("Error: Could not open video file.")
        return
    # Create folder to save frames if it doesn't exist

    # Initialize variables
    frame_count = start_frame
    prev = None
    prev_elements_gray = None
    for _ in range(start_frame):
          video_capture.read()
    while frame_count <= end_frame:
        # Read next frame
        ret, frame = video_capture.read()

        # Break the loop if no more frames or reached end frame
        if not ret or frame_count > end_frame:
            break
        # Resize frame to target resolution (1920x1080)
        target_width = 1920
        target_height = 1080
        frame_resized = cv2.resize(frame, (target_width, target_height))
        # Extract and process the group name region
        x1, y1, x2, y2 = group_name_bbox
        group_name_roi = frame_resized[y1:y2, x1:x2]  # Crop the group name region
        group_name_gray = preprocess_for_ocr(group_name_roi)
        # Extract and process the group elements region
        x1_elem, y1_elem, x2_elem, y2_elem = group_elements_bbox
        group_elements_roi = frame_resized[y1_elem:y2_elem, x1_elem:x2_elem]
        group_elements_gray = cv2.cvtColor(group_elements_roi, cv2.COLOR_BGR2GRAY)
        # Check if there is a significant difference in the group elements region
        if frame_count > start_frame:
            # Compute absolute difference between current and previous group elements frames
            elements_diff = cv2.absdiff(group_elements_gray, prev_elements_gray)
            # Threshold the difference image to detect significant changes
            _, elements_thresh = cv2.threshold(elements_diff, 30, 255, cv2.THRESH_BINARY)

            # Count the number of white pixels (changes) in the thresholded image
            num_changes_elements = np.sum(elements_thresh > 0)
            # Perform OCR on the group name region if significant changes are detected
            if num_changes_elements > 0:
                group_name_text = name_reader.readtext(group_name_roi)
                detected_group_name_text = group_name_text[0][1] if group_name_text else None
                # Save the group elements with the detected group name text


                #coming back here
                if detected_group_name_text:
                  stock_value = get_blue_text(frame)

                  if stock_value not in  location.keys():
                        location[stock_value]=1
                        new_row = pd.DataFrame({'stock code': [stock_value]})
                        df = pd.concat([df, new_row])
                  else:
                    location[stock_value]+=1
                  enlarged_elements_roi = cv2.resize(group_elements_roi, None, fx=3, fy=3, interpolation=cv2.INTER_LINEAR)
                  #extracted_text = reader_en_ch.readtext(enlarged_elements_roi)
                  for bbox in bounding_boxes:
                    # Extract the coordinates of the bounding box
                    pts = bbox
                    # Create a mask for the bounding box region
                    mask = np.zeros(enlarged_elements_roi.shape[:2], dtype=np.uint8)
                    cv2.drawContours(mask, [np.array(pts)], -1, (255, 255, 255), thickness=cv2.FILLED)

                    # Apply the mask to extract the region of interest (ROI)
                    roi = cv2.bitwise_and(enlarged_elements_roi, enlarged_elements_roi, mask=mask)

                    # Convert the ROI to grayscale
                    gray_roi = preprocess_for_ocr(roi)

                    # Perform OCR on the grayscale ROI
                    extracted_text = reader_en_ch.ocr(gray_roi)
                    try:

                      if is_green_text(roi) and '-' and '~' not in extracted_text[0][0][1][0]:
                       results.append('-' + extracted_text[0][0][1][0])
                      else:
                       results.append(extracted_text[0][0][1][0])
                    except:
                      try:
                        if results[0] in target_times:
                          detected_digit = detect_digit_from_ocr(roi)
                          results.append(detected_digit)
                      except:
                          location[stock_value]-=1
                          results=[]
                          break
                  try:
                    if results[0] in target_times:

                      x = [[136, 493], [181, 493], [181, 535], [136, 535]]
                      mask = np.zeros(enlarged_elements_roi.shape[:2], dtype=np.uint8)
                      cv2.drawContours(mask, [np.array(x)], -1, (255, 255, 255), thickness=cv2.FILLED)
                      roix = cv2.bitwise_and(enlarged_elements_roi, enlarged_elements_roi, mask=mask)
                      gray_roix = preprocess_for_ocr(roix)
                      chinese = chinese_reader.readtext(gray_roix)
                      if chinese and chinese[0][1] =='亿':
                        results[4]=str(float(results[4])*10000)
                      results.append(clean_and_convert_to_float(results[1])*clean_and_convert_to_float(results[5]))
                      for i in range(len(results)):
                        df.loc[df['stock code'] == stock_value, list1[i]+str(location[stock_value])] = results[i]
                  except:
                    pass
        prev_elements_gray = group_elements_gray.copy()
        frame_count += 1
        results=[]

    # Release video capture object
    video_capture.release()
    print("Frame processing with OCR completed.")
    return location,df

In [ ]:
import os
import json
from shutil import copyfile
import traceback
def process_video_chunks(video_path, group_name_bbox, group_elements_bbox, bounding_boxes, chunk_duration=30):
    # Extract the base name of the video file (without extension)
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Create the output folder based on the video file name
    output_folder = os.path.join(os.path.dirname(video_path), f"processed_{video_name}")
    print(output_folder)
    os.makedirs(output_folder, exist_ok=True)
    file_path = os.path.join(output_folder, "location.json")
    print(file_path)
    # Split the video into chunks
    chunks = split_video_into_chunks(video_path, chunk_duration)
    processed_chunks = {}
    if os.path.exists(file_path):
        # If the file exists, load its contents into the 'location' dictionary
        with open(file_path, 'r') as file:
            location = json.load(file)
    else:
        # If the file does not exist, initialize 'location' as an empty dictionary
        location = {}
    # Check for existing processed chunk folders
    processed_folders = [folder for folder in os.listdir(output_folder) if folder.startswith("processed_chunk_")]
    if processed_folders:
        processed_chunks = {int(folder.split("_")[-1]): folder for folder in processed_folders}
        print(f"Resuming from processed chunks: {list(processed_chunks.keys())}")

    # Prepare an empty DataFrame to collect results
    all_results = pd.DataFrame(columns=['stock code'])
    for i, chunk in enumerate(chunks):
        if i in processed_chunks:
            # Load the result of the processed chunk
            result_path = os.path.join(output_folder, processed_chunks[i], "result.csv")
            if os.path.exists(result_path):
                processed_result = pd.read_csv(result_path,  sep=';',)

                # Set all_results to the processed_result for the current chunk
                all_results = processed_result.copy()  # Replace all_results with processed_result

                print(f"Adding result of chunk {i} from previous processing.")
        else:
            try:
                location,all_results = process_frames_with_ocr(video_path,chunk[0],chunk[1],group_name_bbox, group_elements_bbox, bounding_boxes,all_results,locations=location)

                chunk_output_folder = os.path.join(output_folder, f"processed_chunk_{i}")
                os.makedirs(chunk_output_folder, exist_ok=True)
                    # Save the chunk_result DataFrame to CSV in chunk_output_folder
                result_csv_path = os.path.join(chunk_output_folder, "result.csv")
                with open(file_path, 'w') as file:
                  json.dump(location, file)
                all_results.to_csv(result_csv_path, sep=';', index=False)

                print(f"Chunk {i} processed successfully and result saved to {result_csv_path}.")
            except Exception as e:
                traceback.print_exc()
                print(f"Error processing chunk {i}: {e}")
                    # Handle error or log the error message

        # Concatenate all collected results into a single DataFrame
    return all_results


# Run

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import time  # Import time for simulating processing

# Define function to start processing video
def start_processing_video(video_path, output_file_name):
    if video_path:
        if output_file_name:
            # Define other parameters (e.g., bounding boxes, threshold)
            bounding_boxes = [
                [[0, 27], [181, 27], [181, 62], [0, 62]],
                [[0, 142], [180, 142], [180, 182], [0, 182]],
                [[20, 260], [177, 260], [177, 297], [20, 297]],
                [[3, 377], [181, 377], [181, 417], [3, 417]],
                [[3, 493], [138, 493], [138, 535], [3, 535]],
                [[3, 611], [181, 611], [181, 650], [3, 650]]
            ]
            group_name_bbox = (1690, 20, 1750, 45)
            group_elements_bbox = (228, 60, 290, 290)
            threshold = 20

            # Call the process_video_chunks function with user input
            df = process_video_chunks(video_path, group_name_bbox, group_elements_bbox,bounding_boxes,20)
            df = process_data(df)
            df = df.apply(extract_ordered_values, axis=1)
            df = sorted_columns(df)
            excel_file_name = f"{output_file_name}.xlsx"
            df = export_to_excel(df, excel_file_name)
        else:
            print("Please provide a valid output file name.")
    else:
        print("Please provide a valid video path.")

video_path = input("Enter video path: ").strip()
output_file_name = input("Enter output file name: ").strip()

# Call the function with provided inputs
start_processing_video(video_path, output_file_name)



Streaming output truncated to the last 5000 lines.
[2024/06/22 05:12:02] ppocr DEBUG: rec_res num  : 1, elapsed : 0.011495351791381836
[2024/06/22 05:12:02] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.018332481384277344
[2024/06/22 05:12:02] ppocr DEBUG: cls num  : 1, elapsed : 0.009149551391601562
[2024/06/22 05:12:02] ppocr DEBUG: rec_res num  : 1, elapsed : 0.012447834014892578
[2024/06/22 05:12:02] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.017951011657714844
[2024/06/22 05:12:02] ppocr DEBUG: cls num  : 1, elapsed : 0.00842738151550293
[2024/06/22 05:12:02] ppocr DEBUG: rec_res num  : 1, elapsed : 0.011797904968261719
[2024/06/22 05:12:02] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.018291711807250977
[2024/06/22 05:12:02] ppocr DEBUG: cls num  : 1, elapsed : 0.009752750396728516
[2024/06/22 05:12:02] ppocr DEBUG: rec_res num  : 1, elapsed : 0.011116743087768555
[2024/06/22 05:12:02] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.017595529556274414
[2024/06/22 05:12:02] ppocr DEBUG: cls

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>